#Semantic Similarity with BERT


This notebook demonstrates semantic similarity which determines how similar
two sentences are, in terms of their meaning. This also demonstrates the use of SNLI (Stanford Natural Language Inference) Corpus for predicting sentence semamntic similarity with transformers.


In [ ]:
# Install transformers library
!pip install transformers

     |████████████████████████████████| 1.3MB 9.0MB/s 
     |████████████████████████████████| 2.9MB 23.5MB/s 
     |████████████████████████████████| 1.1MB 49.1MB/s 
     |████████████████████████████████| 890kB 50.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=6885c57bba2d7339c14a6f127af75d13c024bbb309aec582a15908dc93ea5e21
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
# Import required libraries

import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

## Configuration

In [ ]:
# Define model parameters
max_length = 128  #length of input sentence
batch_size = 32
epoch = 2
label = ["contradiction", "entailment", "neutral"]

## Loading the data

In [ ]:
# Load the data from github
!curl -LO https://raw.githubusercontent.com/MohamadMerchant/SNLI/master/data.tar.gz
!tar -xvzf data.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.1M  100 11.1M    0     0  15.1M      0 --:--:-- --:--:-- --:--:-- 15.1M
SNLI_Corpus/
SNLI_Corpus/snli_1.0_dev.csv
SNLI_Corpus/snli_1.0_train.csv
SNLI_Corpus/snli_1.0_test.csv


## Dataset Overview

Sentence1: The premise caption </br>
Sentence2: The hypothesis caption that was written by the author of the pair.</br>
Similarity: This is the label chosen by the majority of annotators.</br>
The "Similarity" label values in the dataset:</br>
Contradiction: The sentences share no similarity.</br>
Entailment: The sentences have similar meaning.</br>
Neutral: The sentences are neutral.</br>

In [ ]:
# Read data from files.
train_dataset = pd.read_csv("SNLI_Corpus/snli_1.0_train.csv", nrows=100000)
valid_dataset = pd.read_csv("SNLI_Corpus/snli_1.0_dev.csv")
test_dataset = pd.read_csv("SNLI_Corpus/snli_1.0_test.csv")

# Shape of the data
print("Total train samples : {}".format(train_dataset.shape[0]))
print("Total validation samples: {}".format(valid_dataset.shape[0]))
print("Total test samples: {}".format(valid_dataset.shape[0]))

Total train samples : 100000
Total validation samples: 10000
Total test samples: 10000


In [ ]:
print("Sentence1: {}".format(train_dataset.loc[1, 'sentence1']))
print("Sentence2: {}".format(train_dataset.loc[1, 'sentence2']))
print("Similarity: {}".format(train_dataset.loc[1, 'similarity']))

Sentence1: A person on a horse jumps over a broken down airplane.
Sentence2: A person is at a diner, ordering an omelette.
Similarity: contradiction


## Data Preprocessing

In [ ]:
# Drop NaN values present in dataset
print("Number of missing values")
print(train_dataset.isnull().sum())
train_dataset.dropna(axis=0, inplace=True)

Number of missing values
similarity    0
sentence1     0
sentence2     3
dtype: int64


In [ ]:
print("Train Target Distribution")
print(train_dataset.similarity.value_counts())

Train Target Distribution
entailment       33384
contradiction    33310
neutral          33193
-                  110
Name: similarity, dtype: int64


In [ ]:
print("Validation Target Distribution")
print(valid_dataset.similarity.value_counts())

Validation Target Distribution
entailment       3329
contradiction    3278
neutral          3235
-                 158
Name: similarity, dtype: int64


In [ ]:
train_dataset = (
    train_dataset[train_dataset.similarity != "-"]
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)
valid_dataset = (
    valid_dataset[valid_dataset.similarity != "-"]
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)

In [ ]:
# One hot encoding
train_dataset["label"] = train_dataset["similarity"].apply(lambda x: 0 if x == "contradiction" else 1 if x == "entailment" else 2)
y_train = tf.keras.utils.to_categorical(train_dataset.label, num_classes=3)

valid_dataset["label"] = valid_dataset["similarity"].apply(lambda x: 0 if x == "contradiction" else 1 if x == "entailment" else 2)
y_val = tf.keras.utils.to_categorical(valid_dataset.label, num_classes=3)

test_dataset["label"] = test_dataset["similarity"].apply(lambda x: 0 if x == "contradiction" else 1 if x == "entailment" else 2)
y_test = tf.keras.utils.to_categorical(test_dataset.label, num_classes=3)

## Custom data generator creation

In [ ]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    def __init__(
        self,
        sentence_pair,
        label,
        batch_size=batch_size,
        shuffle=True,
        include_target=True,
    ):
        self.sentence_pair = sentence_pair
        self.label = label
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_target = include_target
        # Use base-base-uncased pretrained model in BERT tokenizer to encode the text
        self.tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
        self.indexes = np.arange(len(self.sentence_pair))
        self.on_epoch_end()

    def __len__(self):
        # Number of batches per epoch
        return len(self.sentence_pair) // self.batch_size

    def __getitem__(self, idx):
        # Used for retrieval of the batch of index.
        index = self.index[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pair = self.sentence_pair[index]
        # Encode sentences using BERT tokenizer
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pair.tolist(),
            add_special_token=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_id=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )
        # Conversion of encoded features to numpy array
        input_id = np.array(encoded["input_ids"], dtype="int32")
        attention_mask = np.array(encoded["attention_mask"], dtype="int32")
        token_type_id = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_target:
            label = np.array(self.labels[index], dtype="int32")
            return [input_id, attention_mask, token_type_id], label
        else:
            return [input_id, attention_mask, token_type_id]

    def on_epoch_end(self):
        # If shuffle is set to True shuffle indexes after each epoch
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.index)

## Build the model

In [ ]:
# Create the model under a distribution strategy scope
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Input ids are encoded token ids from BERT tokenizer
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
    # Attention masks indicates which tokens should be attended to
    attention_masks = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="attention_masks")
    # Binary masks identifying different sequences in the model
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="token_type_ids")
    # Load pretrained BERT model
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    bert_model.trainable = False

    sequence_op, pooled_op = bert_model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)
    # Add trainable layers to adapt the pretrained features on the new data
    bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(sequence_op)
    # Apply hybrid pooling approach to bi_lstm sequence output
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concatenate = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concatenate)
    op = tf.keras.layers.Dense(3, activation="softmax")(dropout)
    model = tf.keras.models.Model(inputs=[input_ids, attention_masks, token_type_ids], outputs=op)
    # compile model
    model.compile( optimizer=tf.keras.optimizers.Adam(),loss="categorical_crossentropy", metrics=["acc"],)
    
print("Strategy: {}".format(strategy))
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Strategy: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7fab8cd54668>
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 128, 768), ( 109482240   input_ids[0][0]     

## Train the Model

In [ ]:
train_data = BertSemanticDataGenerator(
    train_dataset[["sentence_1", "sentence_2"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    valid_dataset[["sentence_1", "sentence_2"]].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)

In [ ]:
# Train model
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch 1/2
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
3121/3121 [==============================] - 1086s 348ms/step - loss: 0.6874 - acc: 0.7061 - val_loss: 0.5240 - val_acc: 0.7932
Epoch 2/2
3121/3121 [==============================] - 1079s 346ms/step - loss: 0.5885 - acc: 0.7596 - val_loss: 0.4969 - val_acc: 0.8087


## Fine-tuning

In [ ]:
# Unfreeze the bert_model for finetuning
bert_model.trainable = True
# Recompile the model to ensure the change effective
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 128, 768), ( 109482240   input_ids[0][0]                  
                                                                 attention_masks[0][0] 

## Train the entire model end-to-end

In [ ]:
# Training the entire model
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epoch,
    use_multiprocessing=True,
    workers=-1,
)

Epoch 1/2


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


2111/3121 [===================>..........] - ETA: 14:33 - loss: 0.4842 - accuracy: 0.8091